In [1]:
from datetime import datetime
import random
import routingpy as rp
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd

## Isochrones

In [2]:
V = rp.Valhalla("http://valhalla_hist_traffic-valhalla-1:8002")
START_TIMES = [(n*24*60*60) + 1646625600 for n in range(5)]
END_TIMES = [n + 19 * 60 * 60 for n in START_TIMES]
LOCATIONS = [
    [13.4101, 52.5226],  # Alex
    [13.434992,52.480835],  # Neukölln
    [13.495309,52.545576],  # Alt-Hohenschönh.
    [13.351479,52.550044],  # Wedding  
    [13.330997,52.487294]  # Wilmersdorf
]

In [3]:
def strf_epoch(e: int) -> str:
    return datetime.utcfromtimestamp(e).isoformat()[:-3]

In [4]:
def time_range(start_epoch: int, end_epoch: int, step: int = 300):
    for time_ in range(start_epoch, end_epoch, step):
        yield {"type": 1, "value": strf_epoch(time_)}

In [12]:
isochrones_data = []
point_ix = 0
for location in LOCATIONS:
    for START_TIME, END_TIME in zip(START_TIMES, END_TIMES):
        for date_time in time_range(START_TIME, END_TIME, 900):
            isochrones = V.isochrones(locations=location, profile="auto", intervals=[100, 250, 500], date_time=date_time)
            interval_ix = 0
            for isochrone in isochrones:
                iso_geom = Polygon(isochrone.geometry)
                isochrones_data.append({"data": {"time": date_time["value"], "center": Point(*location), "loc_id": point_ix, "interval_id": interval_ix}, "geometry": iso_geom})
                interval_ix += 1
    point_ix += 1

In [13]:
iso_df = gpd.GeoDataFrame(data=[i["data"] for i in isochrones_data] ,geometry=[i["geometry"] for i in isochrones_data], crs="EPSG:4326")

In [14]:
iso_df_polys = iso_df.drop(columns=["center"])

In [15]:
with open("./isochrones.geojson", "w") as fh:
    fh.write(iso_df_polys.to_json())

In [16]:
with open("./points.geojson", "w") as fh:
    fh.write(gpd.GeoDataFrame(iso_df["center"].drop_duplicates()).rename(columns={"center": "geometry"}).to_json())

## Directions

In [5]:
gdf = gpd.read_file("./adressen_berlin.geojson")

In [6]:
LOCATIONS = [
 [13.4101, 52.5226],  # Alex
    *[row.geometry[0].coords[0] for index, row in gdf.sample(500, random_state=1).iterrows()]
]
del gdf

In [7]:
point_ix = 0
for file_index, (START_TIME, END_TIME) in enumerate(zip(START_TIMES, END_TIMES)):
    directions_data = []
    for date_time in (time_range(START_TIME, END_TIME, 900)):
        for location in LOCATIONS[1:]:
            directions = V.directions(locations=[location, LOCATIONS[0]], profile="auto", intervals=[100, 250, 500], date_time=date_time)
            geom = LineString(directions.geometry)
            directions_data.append({"data":
                                    {
                                        "time": date_time["value"], 
                                        "loc_id": point_ix,
                                        "duration": directions.duration,
                                        "distance": directions.distance
                                    }, 
                                    "geometry": geom})
    directions_df = gpd.GeoDataFrame(data=[d["data"] for d in directions_data] ,geometry=[d["geometry"] for d in directions_data], crs="EPSG:4326")
    directions_df.to_file(f"./shp/directions_{file_index}.shp")
    point_ix += 1

In [8]:
dir_points_df = gpd.GeoDataFrame(geometry=[Point(*l) for l in LOCATIONS], crs="EPSG:4326")

In [9]:
with open("./directions_locations.geojson", "w") as fh:
    fh.write(dir_points_df.to_json())

In [11]:
del directions_df